In [2]:
%pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 435.7 kB/s eta 0:01:30
   ---------------------------------------- 0.1/38.8 MB 751.6 kB/s eta 0:00:52
   ---------------------------------------- 0.2/38.8 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.3/38.8 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.5/38.8 MB 1.7 MB/s eta 0:00:24
    --------------------------------------- 0.7/38.8 MB 2.1 MB/s eta 0:00:18
    --------------------------------------- 0.8/38.8 MB 2.3 MB/s eta 0:00:17
    --------------------------------------- 0.8/38.8 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.3/38.8 MB 3.0 MB/s eta 0:00:13
   - -------------------------------------- 1.3/38.8 MB 3.0 MB/s eta 0:00:13
   - -------------------------------------- 1.6/38.8 MB 2.8 MB/s eta 0:00:14
   - ----


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2

image_path = 'Lake.jpg'
image = cv2.imread(image_path)
image.shape

(494, 480, 3)

In [3]:
import matplotlib.pyplot as plt

cv2.imshow("Image",image)

: 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.metrics import silhouette_samples, silhouette_score
from tqdm import tqdm

In [ ]:
#load image
image_path = 'photo2.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
print(image.shape)
plt.imshow(image)
plt.show()


In [ ]:
pixels = image.reshape(-1,3)
pixels.shape
num_clusters = 4
kmeans = KMeans(n_clusters = num_clusters, random_state= 42)
kmeans.fit(pixels)
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_
np.rint(cluster_centers)

clustered_image = cluster_centers[labels].astype(np.uint8).reshape(image.shape)
plt.imshow(clustered_image)

plt.imshow(image)
plt.show()

In [ ]:
cluster_sizes = np.bincount(labels, minlength = num_clusters)
total_pixels = labels.size
percentage_pixels = (cluster_sizes /total_pixels)*100
plt.pie(percentage_pixels,colors=np.array(cluster_centers/255),labels=[f"Cluster{i}" for i in range(num_clusters)], autopct = '%.2f%%')
plt.show()

cluster_data ={
    'Cluster':range(num_clusters),
    'Pixel_Count' : cluster_sizes,
    'Red':cluster_centers[:,0],
    'Blue':cluster_centers[:,1],
    'Green':cluster_centers[:,2]
}
cluster_dataframe = pd.DataFrame(cluster_data)

cluster_dataframe.set_index('Cluster', inplace = True)
cluster_dataframe

In [ ]:
pixel_sum = sum(cluster_dataframe['Pixel_Count'])
print(int(cluster_dataframe['Pixel_Count'][1]/pixel_sum * 200000),'Sq.KM')
# Calculate WCSS for different k values
wcss = []
for k in range(2,15):
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(pixels)
    wcss.append(kmeans.inertia_)
# Plot the elbow method graph
plt.plot(range(2, 15), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

In [ ]:

n_clusters = range(3,5)
for num_clusters in n_clusters:
    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(pixels)

    # Calculate silhouette score
    silhouette_avg = silhouette_score(pixels, labels)
    print(f"Silhouette Score for {num_clusters} clusters: {silhouette_avg}")

    # Calculate silhouette values for each data point
    silhouette_values = silhouette_samples(pixels, labels)

type(silhouette_values)


In [ ]:
# Create a silhouette plot
plt.figure(figsize=(8, 6))
y_lower = 10
for i in range(num_clusters):
    cluster_silhouette_values = silhouette_values[labels == i]
    cluster_silhouette_values.sort()
    cluster_size = cluster_silhouette_values.shape[0]
    y_upper = y_lower + cluster_size

    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_values, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * cluster_size, str(i))

    y_lower = y_upper + 10
    
plt.xlabel("Silhouette coefficient values")
plt.ylabel("Cluster label")
plt.title(f"Silhouette plot for {num_clusters} clusters")
plt.axvline(x=silhouette_avg, color="red", linestyle="--")
plt.yticks([])
plt.show()

In [ ]:

# Create a histogram of silhouette values
plt.hist(silhouette_values, bins=20)
plt.xlabel("Silhouette coefficient values")
plt.ylabel("Frequency")
plt.title(f"Histogram of Silhouette Coefficients for {num_clusters} clusters")
plt.show()